# Mink usage examples

This notebook demonstrates some examples of using mink and how it interacts with sklearn.

## Imports

In [1]:
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
import tensorflow as tf
import pandas as pd

In [2]:
from mink import NeuralNetClassifier
from mink import NeuralNetRegressor
from mink.layers import DenseLayer
from mink.layers import InputLayer
from mink.updates import SGD

## MNIST classification task

### Data

In [3]:
X, y = make_classification(n_samples=5000)

### Define neural net

Note: No need to specify the shape of the training data, number of classes, or to set softmax nonlinearity. The `NeuralNetClassifier` class takes care of all of that, as is expected from an sklearn estimator.

In [5]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [6]:
net = NeuralNetClassifier(l2, verbose=1)

If we want to change certain parameters after initialization, just use the `set_params` method and the double-underscore notation known from sklearn.

In [7]:
net.set_params(update__learning_rate=0.5)

NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa44c724e48>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

### Train

In [8]:
net.fit(X, y, num_epochs=0)

NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa44c724e48>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [9]:
y_proba = net.predict_proba(X)

In [10]:
(np.argmax(y_proba, 1) == y).mean()

0.52700000000000002

In [11]:
pdb on

Automatic pdb calling has been turned ON


In [12]:
net.fit(X, y, num_epochs=20)

epochs:    1 | loss: 0.42709
epochs:    2 | loss: 0.41273
epochs:    3 | loss: 0.40178
epochs:    4 | loss: 0.39372
epochs:    5 | loss: 0.38829
epochs:    6 | loss: 0.38415
epochs:    7 | loss: 0.38011
epochs:    8 | loss: 0.37595
epochs:    9 | loss: 0.37180
epochs:   10 | loss: 0.36706
epochs:   11 | loss: 0.36050
epochs:   12 | loss: 0.35401
epochs:   13 | loss: 0.34827
epochs:   14 | loss: 0.34199
epochs:   15 | loss: 0.33549
epochs:   16 | loss: 0.33029
epochs:   17 | loss: 0.32476
epochs:   18 | loss: 0.31914
epochs:   19 | loss: 0.31401
epochs:   20 | loss: 0.30976


NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa44c724e48>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [13]:
y_proba

array([[ 0.37726831,  0.62273169],
       [ 0.29548568,  0.70451432],
       [ 0.37938896,  0.62061101],
       ..., 
       [ 0.41983604,  0.58016396],
       [ 0.41554657,  0.5844534 ],
       [ 0.48238996,  0.51761007]], dtype=float32)

In [14]:
np.argmax(y_proba, 1)[:10]

array([1, 1, 1, 0, 1, 1, 1, 1, 0, 0])

In [15]:
y_proba = net.predict_proba(X)

In [16]:
(np.argmax(y_proba, 1) == y.flatten()).mean()

0.89219999999999999

### Grid search

The neural net estimators can be used in conjunction with other sklearn features, such as `GridSearchCV`.

In [17]:
l0 = InputLayer()
l1 = DenseLayer(l0, name='hidden')
l2 = DenseLayer(l1)

In [18]:
net = NeuralNetClassifier(l2, update=SGD())

Parameters are set using the known double-underscore notation, e.g.

`'update__learning_rate': [0.1, 0.5]`.

Note: Instead of having to write

`'layer__incoming__num_units': [50, 100]`

we can just write

`'hidden__num_units': [50, 100]`

because we have given the hidden layer a name, "hidden". This may safe a lot of writing and confusion.

In [19]:
params = {
    'update__learning_rate': [0.1, 0.5],
    'max_epochs': [5, 10],
    'hidden__num_units': [50, 100],
}

In [20]:
cv = GridSearchCV(net, params, scoring='accuracy', refit=False, verbose=3)

In [21]:
cv.fit(X, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] update__learning_rate=0.1, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.1, max_epochs=5, hidden__num_units=50, score=0.850630 -   0.2s
[CV] update__learning_rate=0.1, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.1, max_epochs=5, hidden__num_units=50, score=0.832633 -   0.3s
[CV] update__learning_rate=0.1, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.1, max_epochs=5, hidden__num_units=50, score=0.827731 -   0.4s
[CV] update__learning_rate=0.5, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.5, max_epochs=5, hidden__num_units=50, score=0.855429 -   0.3s
[CV] update__learning_rate=0.5, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.5, max_epochs=5, hidden__num_units=50, score=0.840432 -   0.4s
[CV] update__learning_rate=0.5, max_epochs=5, hidden__num_units=50 ...
[CV]  update__learning_rate=0.5, max_epochs=5, hi

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   16.7s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa44c724e48>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=Glor...e=CrossEntropy(eps=1e-12),
          session_kwargs=None, update=SGD(learning_rate=0.01), verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'update__learning_rate': [0.1, 0.5], 'max_epochs': [5, 10], 'hidden__num_units': [50, 100]},
       pre_dispatch='2*n_jobs', refit=False, scoring='accuracy', verbose=3)

In [22]:
cv.best_params_, cv.best_score_

({'hidden__num_units': 50, 'max_epochs': 10, 'update__learning_rate': 0.5},
 0.84719999999999995)

## Regression task

As is known from sklearn, we have separate estimators for classification and regression.

### Data

In [23]:
X, y = make_regression(n_samples=5000)

Note that apart from using `NeuralNetRegressor` instead of `NeuralNetClassifier`, everything is the same. No need to adjust output nonlinearity or objective.

### Define neural network

In [25]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [26]:
net = NeuralNetRegressor(l2, verbose=1)

In [27]:
net.set_params(update__learning_rate=0.001)

NeuralNetRegressor(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa44c72a278>,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=MeanSquaredError(), session_kwargs=None,
          update=SGD(learning_rate=0.001), verbose=1)

### Train

In [28]:
net.fit(X, y, num_epochs=20)

epochs:    1 | loss: 3603.20703
epochs:    2 | loss: 240.22165
epochs:    3 | loss: 178.11111
epochs:    4 | loss: 135.71141
epochs:    5 | loss: 103.19876
epochs:    6 | loss: 78.38826
epochs:    7 | loss: 59.37325
epochs:    8 | loss: 45.04291
epochs:    9 | loss: 34.18863
epochs:   10 | loss: 26.17146
epochs:   11 | loss: 20.21072
epochs:   12 | loss: 15.72537
epochs:   13 | loss: 12.32325
epochs:   14 | loss: 9.71921
epochs:   15 | loss: 7.76906
epochs:   16 | loss: 6.31207
epochs:   17 | loss: 5.21556
epochs:   18 | loss: 4.37478
epochs:   19 | loss: 3.72983
epochs:   20 | loss: 3.22991


NeuralNetRegressor(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa44c72a278>,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Linear(), num_units=1),
          max_epochs=10, objective=MeanSquaredError(), session_kwargs=None,
          update=SGD(learning_rate=0.001), verbose=1)